
# 漢字・アルファベットからカタカナを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [ ]:
# スクリプトで使う環境変数をセット

import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        ver=l.rstrip()
        break

os.environ["VER"]=ver
os.environ['DATA']='../dict/data/data.'+os.environ["VER"]
os.environ['DIR']='dataset/dataset.'+os.environ["VER"]
os.environ['LOG']='logs/logs.'+os.environ["VER"]
os.environ['MODEL']='model/model.'+os.environ["VER"]


In [2]:
!echo $VER

1.7o


In [3]:

!mkdir -p $DIR
!mkdir -p $LOG
!mkdir -p $MODEL

In [4]:
# オープンソース辞書とWikipediaから作成した単語辞書をマージする
!python mergedic.py --dic1 $DATA/reliable_$VER.json --dic2 $DATA/tankanji_$VER.json --outfile tmp.json



{
  "dic1": "../dict/data/data.1.7o/reliable_1.7o.json",
  "dic2": "../dict/data/data.1.7o/tankanji_1.7o.json",
  "outfile": "tmp.json"
}
len=2026891
cnt=2215792


In [5]:
# 作成したデータを訓練用，開発用，検証用に分ける。開発用と検証用がそれぞれ全体の0.01
!python prep.py --json tmp.json --outdir $DIR --ratio 0.01


{
  "json": "tmp.json",
  "outdir": "dataset/dataset.1.7o",
  "ratio": 0.01,
  "reverse": false
}
k_max_len=72
v_max_len=120
train,2171478
valid,22157
test,22157


In [6]:
# ひらがなとカタカナのペアデータを作成し，訓練データに混ぜる
!python kana.py --outfile $DIR/hirakata.txt

!cat $DIR/hirakata.txt |awk -F, '{print $1}'>> $DIR/train.src
!cat $DIR/hirakata.txt |awk -F, '{print $2}'>> $DIR/train.tgt

{
  "outfile": "dataset/dataset.1.7o/hirakata.txt",
  "reverse": false
}


In [7]:
# 単漢字とカタカナのデータを，訓練データに混ぜる 

!python json2txt.py --json $DATA/tankanji_$VER.json --txt tmp.txt

!cat tmp.txt |awk -F, '{print $1}' >> $DIR/train.src
!cat tmp.txt |awk -F, '{print $2}' >> $DIR/train.tgt
!rm -rf tmp.txt tmp.json

{
  "json": "../dict/data/data.1.7o/tankanji_1.7o.json",
  "txt": "tmp.txt"
}


In [8]:
# 開発，検証用に，訓練データに入っているものがあれば，削除する(インサンプルを阻止)
!python omit_testval.py --train_src $DIR/train.src --train_tgt $DIR/train.tgt --test_src $DIR/test.src --test_tgt $DIR/test.tgt --valid_src $DIR/valid.src --valid_tgt $DIR/valid.tgt



{
  "train_src": "dataset/dataset.1.7o/train.src",
  "train_tgt": "dataset/dataset.1.7o/train.tgt",
  "test_src": "dataset/dataset.1.7o/test.src",
  "test_tgt": "dataset/dataset.1.7o/test.tgt",
  "valid_src": "dataset/dataset.1.7o/valid.src",
  "valid_tgt": "dataset/dataset.1.7o/valid.tgt"
}
train=2286056
omit=2195


In [9]:
# 訓練，開発，検証データをシャッフルする

!python shuffle.py --src $DIR/train.src --tgt $DIR/train.tgt
!python shuffle.py --src $DIR/valid.src --tgt $DIR/valid.tgt
!python shuffle.py --src $DIR/test.src --tgt $DIR/test.tgt



{
  "tgt": "dataset/dataset.1.7o/train.tgt",
  "src": "dataset/dataset.1.7o/train.src"
}
{
  "tgt": "dataset/dataset.1.7o/valid.tgt",
  "src": "dataset/dataset.1.7o/valid.src"
}
{
  "tgt": "dataset/dataset.1.7o/test.tgt",
  "src": "dataset/dataset.1.7o/test.src"
}


In [10]:
# スペース区切りにする
!python space.py --infile $DIR/train.src --outfile $DIR/train.src
!python space.py --infile $DIR/train.tgt --outfile $DIR/train.tgt
!python space.py --infile $DIR/valid.src --outfile $DIR/valid.src
!python space.py --infile $DIR/valid.tgt --outfile $DIR/valid.tgt
!python space.py --infile $DIR/test.src --outfile $DIR/test.src
!python space.py --infile $DIR/test.tgt --outfile $DIR/test.tgt

{
  "infile": "dataset/dataset.1.7o/train.src",
  "outfile": "dataset/dataset.1.7o/train.src"
}
{
  "infile": "dataset/dataset.1.7o/train.tgt",
  "outfile": "dataset/dataset.1.7o/train.tgt"
}
{
  "infile": "dataset/dataset.1.7o/valid.src",
  "outfile": "dataset/dataset.1.7o/valid.src"
}
{
  "infile": "dataset/dataset.1.7o/valid.tgt",
  "outfile": "dataset/dataset.1.7o/valid.tgt"
}
{
  "infile": "dataset/dataset.1.7o/test.src",
  "outfile": "dataset/dataset.1.7o/test.src"
}
{
  "infile": "dataset/dataset.1.7o/test.tgt",
  "outfile": "dataset/dataset.1.7o/test.tgt"
}


In [11]:
# フォーマットする
!python format.py --src $DIR/train.src --tgt $DIR/train.tgt --outfile $DIR/train.jsonl
!python format.py --src $DIR/test.src --tgt $DIR/test.tgt --outfile $DIR/test.jsonl
!python format.py --src $DIR/valid.src --tgt $DIR/valid.tgt --outfile $DIR/valid.jsonl



## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead    
マルチヘッド数

- ffn_hid_dim    
FFNのDimension

- num_encoder_layer    
エンコードレイヤーの数

- num_decoder_layer    
デコードレイヤーの数

- lr     
学習率

- dropout    
ドロップアウトの割合, 0-1

- num_epochs    
何周学習データを用いて学習を行うか

- device    
mps,cpu,cudaから選択
  - mps        
   アップルシリコンを搭載したマシンで実行する際に選択
  - cuda         
   CUDAが利用できる環境で選択
  - cpu        
   上記以外は，CPUモードを選択

- earlystop_patient    
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir    
モデルの出力ディレクトリ

- tensorboard_logdir    
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix    
jsonl形式のデータのprefix

- source_lang    
jsonl形式のデータのsourceのキー

- target_lang    
jsonl形式のデータのtargetのキー

- train_file    
訓練用データのJSONLファイル

- valid_file    
開発用データのJSONLファイル

In [18]:
# 訓練実行
!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 64 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


[00:00:08] Pre-processing sequences       ██████████████████ 0        /        0[00:00:00] Pre-processing sequences       ██████████████████ 0        /        0/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
num_epochs:100
/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch: 1, Train loss: 2.372, Val loss: 1.400, Epoch time = 6528.177s
Epoch: 2, Train loss: 1.378, Val loss: 0.843, Epoch time = 6507.484s
Epoch: 3, Train loss: 1.010, Val loss: 

In [13]:
%load_ext tensorboard
%tensorboard --logdir logs/logs.$VER

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file
jsonl形式の検証用データファイル

- model_file
transformer_model.pyで作成したモデルファイル。

- outfile
検証結果を格納するファイル

- device
cpu限定

- nbest
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len
出力するカタカナ姓名の最大長さ

- search
greedy,beam,bothから選択
-- greedy
貪欲法による探索
-- beam
ビームサーチによる探索
-- both
貪欲砲，ビームサーチ両方の探索



In [19]:
# 検証用データを実行
!python ./generate.py \
    --test_file $DIR/test.jsonl \
    --model_file $MODEL/checkpoint_best.pt \
    --outfile $DIR/generate.txt \
    --source_lang kana \
    --target_lang kanji \
    --device cpu \
    --nbest 5 \
    --beam_width 5 \
    --max_len 100 \
    --search beam




/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [20]:
# 検証結果を評価
!python evaluate.py --infile $DIR/generate.txt


dataset/dataset.1.7o/generate.txt,acc=0.911946563162883


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [22]:
!python conv_scripted.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_$VER.pt \
    --encoder=$MODEL/encoder_$VER.pt \
    --decoder=$MODEL/decoder_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_$VER.pt \
    --generator=$MODEL/generator_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_$VER.pt \
    --vocab_src=$MODEL/vocab_src_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_$VER.txt \
    --params=$MODEL/params_$VER.json \
    --device=cpu



ver=1.7o
/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/jit/_recursive.py:312: UserWarning: 'norm' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(


In [23]:
!date

2025年 10月  6日 月曜日 16:16:16 JST
